In [3]:
from sys import path
from os import path as ospath
path.append(ospath.join("..", "..", "src"))

from sequence import Sequence, loadFasta
from aminoacid import AminoAcid
from score import ScoreMatrix
from align import Align
from math import log, ceil

# Project 2 - Substitution matrices

In the past project we've made extensive use of Soring systems in order to achieve sequence alignment. The scores assigned to each amino acid pair were not understood, we simply used standard ones. The scoring systems we used are called **substitution matrices** as they provide scores for each possible amino acid substitution. When comparing two sequences, one of the following is true:
* **Evolutionary model**: sequences B and A do not have a common origin. Therefore B can be seen as completely random when compared to A. This does not mean B has to be completely different than A : amino acids $a$ and $b$ could still align by chance. We can calculate the probability of that happening by looking at the amino acid frequencies $p_a$ and $p_b$ in the population.
$$Prob(a, b) = p_a \cdot p_b $$

* **Random model**: sequences B and A have a common origin. Therefore B is not random, and a good alignment between A and B will provide us with the result of each of their mutations. The probability of amino acids $a$ and $b$ being aligned depends on the likelihood of substitutions from $a$ to $b$ or $b$ to $a$.
$$Prob(a, b) = q_{a,b}$$

The goal of this section is to be able to tell these cases apart. In order to do that, we'll construct substitution matrices, assigning to each amino acid pair a score equal to the **log-odds ratio** $\frac{q_{a,b}}{p_a \cdot p_b}$.

## BLOCKS

`BLOCKS` is a database of amino acid sequences. It was used to construct the standard BLOSUM matrices. We used it to retrieve two families of sequences : the SH3 and PDZ domains. These come in multiple "blocks" of gapless aligned sequences of the same size. We quickly formatted these informations into .fasta files compatible with our previous project. With them, we'll be able to construct our own BLOSUM matrices specific to these domains.

## BLOSUM

`BLOSUM` or BLOcks SUbstitution Matrix uses the well-conserved (gapless and aligned) sequences found in `BLOCKS` to determine the numerical values of $p_a$, $p_b$ and $q_{a,b}$, based on the required identity between them. For example, `BLOSUM 62` is the `BLOSUM` that was calculated for a required identity of 62%. Here's an overview of the algorithm for calculating the log-odds ratios with an identity of $X$% :

* Separate all sequences into $t$ groups $G_{i}$, such that all sequences $s_{j} \in G_{i}$ in each group have at least $X$ percent identity with some other sequence of the same group. We assume the sequences have she same lenght $n$.

In [3]:
def belongs(outSequence, group, minMatches):
	"""
	Returns True if there's at least 'minMatches' matches between outSequence and any sequence 
	from 'group'; False otherwise.
	Size of all sequences is assumed to be equal.
	"""
	for inSequence in group:
		matches = 0
		for i in range(len(outSequence)):
			if outSequence[i] == inSequence[i]:
				matches += 1
			if matches >= minMatches: #As soon as a match is found
				return True
	return False

	
def makeGroupsFromFasta(path, requiredIdentityPercent):
	"""
	Loads Sequences from file at 'path' and separate them in groups with an identity of 
	at least 'requiredIdentityPercent'.
	Returns a list representing the groups as lists of Sequence objects.
	"""
	sequences = [seq for seq in loadFasta(path)]
	groups = [[sequences[0]]] #First sequence is assigned to first group
	
	seqSize = len(sequences[0]) #Size of the sequences
	
	#Number of matches required to achieve requiredIdentityPercent
	minMatches = ceil((requiredIdentityPercent/100)*seqSize)
	
	#For each outSequence not yet in a group
	for outSequence in sequences[1:]:
		groupFound = False #has a group been found ?
		groupIndex = 0 #index of the group we're looking in
		
		#Look for a group where outSequence belongs
		while (not groupFound) and groupIndex < len(groups):
			if belongs(outSequence, groups[groupIndex], minMatches):
				groups[groupIndex].append(outSequence)
				groupFound = True
			else:
				groupIndex += 1 #Move on to next group
		
		#If no group works, create a new one
		if not groupFound:
			groups.append([outSequence])
	
	return groups

def valueDictsFromGroups(groups):
	"""
	Transforms each group from 'groups' into a list of dictionaries, one for each Sequence column.
	The dictionaries map each AminoAcid found in that column to their count.
	Returns the list of dictionaries and a list of the size (in Sequences) of their groups.
	"""
	seqSize = len(groups[0][0])
	groupCount = len(groups)
	
	groupValues = []
	groupSizes = []
	
	for group in groups: #For each group
		groupAAs = []
		groupSize = len(group) #Size of group (n°sequences)
		
		for col in range(seqSize): #For each column
			groupCol = {}
			
			for seq in group: #For each sequence in group
				try:
					groupCol[seq[col]] += 1 #Increment count
				except:
					groupCol[seq[col]] = 1
			
			groupAAs.append(groupCol)
		groupValues.append(groupAAs)
		groupSizes.append(groupSize)
	return groupValues, groupSizes

* Calculate the weighted frequencies of all amino acid pairs, where $Count(G_{i_k}, a)$ is the number of occurrences of amino acid $a$ in column $k$ of sequences from $G_i$, $Size(G_i)$ is the number of sequences in group $G_i$ :

$$f_{a,b} = \sum_{k=1}^{n}{\sum_{i=1}^{t-1}{\sum_{j=i+1}^{t}{ \frac{Count(G_{i_k},a)}{Size(G_i)} \cdot \frac{Count(G_{j_k},b)}{Size(G_j)} + \frac{Count(G_{i_k},b)}{Size(G_i)} \cdot \frac{Count(G_{j_k},a)}{Size(G_j)}}}}$$

In [4]:
def getFrequencies(groupValues, groupSizes):
	"""
	Evaluates the frequencies of AminoAcids within columns of groups in 'groupValues'.
	Frequencies are weighted according to group sizes in 'groupSizes'.
	Returns two dictionaries and a number:
		-'freqPairs' maps pairs of AminoAcids to their frequencies
		-'freqSingle' maps single AminoAcids to their frequencies
		-'freqSum' is the sum of all frequencies
	"""
	seqSize = len(groupValues[0]) #Size of the Sequences
	groupCount = len(groupSizes) #Number of groups
	
	freqPairs = {} #frequencies of amino acid pairs (fAB)
	
	#Frequencies of single amino acids (fA)
	freqSingle = {AminoAcid(aa):0 for aa in AminoAcid.getAllNames()} 
	
	freqSum = 0 #Sum of frequencies  sum(fAB)
	
	for col in range(seqSize): #Each column
		for groupAIndex in range(groupCount-1): #Each groupA
			groupA = groupValues[groupAIndex]
			groupASize = groupSizes[groupAIndex]
			
			for groupBIndex in range(groupAIndex+1, groupCount): #Each further groupB
				groupB = groupValues[groupBIndex]
				groupBSize = groupSizes[groupBIndex]
				
				for aaA, aaACount in groupA[col].items(): #Each AA from groupA
					aaAFreq = aaACount / groupASize #Its frequency within groupA
					
					for aaB, aaBCount in groupB[col].items(): #Each AA from groupB
						aaBFreq = aaBCount / groupBSize #Its frequency within groupB
						
						aaPairFreq = aaAFreq * aaBFreq #Pair frequency
						freqSum += aaPairFreq	#Sum of all frequencies			
						freqSingle[aaA] += aaPairFreq/2
						freqSingle[aaB] += aaPairFreq/2
						
						#Index is unique to this pair
						pairIndex = (aaA, aaB) if aaA > aaB else (aaB, aaA)
						try:
							freqPairs[pairIndex] += aaPairFreq
						except:
							freqPairs[pairIndex] = aaPairFreq
	
	return freqPairs, freqSingle, freqSum

* Calculate the probability of occurrences $a, b$ in the evolutionary model as the frequence of pairs $(a,b)$ divided by the frequence of all possible pairs
$$q_{a,b} = \frac{f_{a,b}}{\sum_{1\leq a \leq b}{f_{a,b}}}$$

* Calculate the probability of occurrences $a, b$ in the random model, based on the observed frequencies of each single amino acid
$$
e_{a,b} =
\left\{
    \begin{array}{ll}
      p_a^2 \quad \textrm{if} \quad a = b\\
      2 p_a p_b \quad \textrm{if} \quad a \neq b
    \end{array}
\right.
\quad
\textrm{where}
\quad
p_a = q_{a,a} + \frac{1}{2}\sum_{a \neq b}{q_{a,b}}
$$

* Compute the log-odds ratio of all probabilities for the evolutionary and random models. This gives us a negative value (a bad score) if the random model is more likely, and a positive value the other way around. This value is the one used in BLOSUM (except we round it to the nearest integer).
$$s_{a,b} = 2 \; \log_{2}{\frac{q_{a,b}}{e_{a,b}}}$$

In [5]:
def sumFrequenciesToProb(freqPairsList, freqSingleList, freqSumList):
	"""
	Sums all frequencies in provided lists, and transforms them to probabilities according
	to the sum of frequencies found in 'freqSumList'.
	"""
	fSum = sum(freqSumList) #Absolute sum of all frequencies
	probPairs, probSingle = {}, {} #Probabilities for pairs and single AAs
	
	for freqPairs, freqSingle in zip(freqPairsList, freqSingleList):
		for key,value in freqPairs.items():
			#Sum all frequencies for matching AA pairs, divided by fSum
			try:
				probPairs[key] += value / fSum 
			except:
				probPairs[key] = value / fSum
		for key,value in freqSingle.items():
			#Sum all frequencies for matching AAs, divided by fSum
			try:
				probSingle[key] += value / fSum
			except:
				probSingle[key] = value / fSum
	
	return probPairs, probSingle
		
	
def blosumFromProbabilities(probPairs, probSingle, requiredIdentityPercent):
	"""
	Fills and returns a Score according to the BLOSUM algorithm, from the probabilities
	of AA pairs and singletons provided in 'probPairs' and 'probSingle'.
	"""
	#Create empty Score, ignoring AAs B,Z,J,U,O
	scoreMatrix = Score("", "BLOSUM{}".format(requiredIdentityPercent), "BZJUO")
	
	for key, qAB in probPairs.items():
		#qAB is the evolutionary probability of the AA pair (A,B)
		aaA, aaB = key #Both amino acids
		pA, pB = probSingle[aaA], probSingle[aaB] #Their single probabilities
		
		#eAB is the random probability of the AA pair (A, B) given their single probabilities
		eAB = pA * pA if aaA == aaB else 2 * pA * pB
		
		#The BLOSUM score for this pair is the log-odds-ratio of evolutionary and random prob.
		sAB = int(round(2 * log(qAB / eAB, 2)))
		scoreMatrix.setScore(aaA, aaB, sAB) #Fill the matrix
	
	return scoreMatrix
			
		
			
def blosumFromFasta(requiredIdentityPercent, *filepaths):
	"""
	Creates and returns a Score for all sequences in the provided 'filepaths',
	using the BLOSUM approach with an identity of at least 'requiredIdentityPercent'.
	Each file is grouped independently and only then their weighted probabilities are merged.
	"""
	#Results for each different files are first stored in lists
	freqPairsList, freqSingleList, freqSumList = [], [], [] 
	
	for path in filepaths: #for each .fasta file
		groups = makeGroupsFromFasta(path, requiredIdentityPercent) #groups
		groupValues, groupSizes = valueDictsFromGroups(groups) #groups as dicts
		freqPairs, freqSingle, freqSum = getFrequencies(groupValues, groupSizes) #frequencies
		
		freqPairsList.append(freqPairs) #Append results
		freqSingleList.append(freqSingle)
		freqSumList.append(freqSum)
	
	#Merge (sum) results together
	probPairs, probSingle = sumFrequenciesToProb(freqPairsList, freqSingleList, freqSumList)
	#Create the BLOSUM matrix
	blosum = blosumFromProbabilities(probPairs, probSingle, requiredIdentityPercent)
	return blosum

In [6]:
sh3Paths = ["SH3-A.fasta", "SH3-B.fasta", "SH3-C.fasta", "SH3-D.fasta"]
print("SH3 Family")
blosum40sh3 = blosumFromFasta(40, *sh3Paths)
print(blosum40sh3)
blosum70sh3 = blosumFromFasta(70, *sh3Paths)
print(blosum70sh3)

print("PDZ Family")
pdzPaths = ["PDZ-A.fasta", "PDZ-B.fasta"]
blosum40pdz = blosumFromFasta(40, *pdzPaths)
print(blosum40pdz)
blosum70pdz = blosumFromFasta(70, *pdzPaths)
print(blosum70pdz)

SH3 Family
---------- BLOSUM40 ----------
A   2   
C   3   6   
D   -1  -10 2   
E   0   -8  2   3   
F   -1  3   -4  -4  4   
G   0   0   3   -1  -3  4   
H   0   -7  -2  0   -5  -3  2   
I   -2  0   -2  -2  2   -2  0   1   
K   1   -8  0   2   -3  -3  1   -1  2   
L   -2  -3  -3  -2  0   -2  0   2   -1  2   
M   -1  -1  -1  -2  2   -5  -1  2   -3  2   0   
N   1   1   1   0   2   1   -2  -3  2   -4  -8  3   
P   0   2   0   0   -5  -1  -2  -3  0   -3  -3  0   5   
Q   -1  -8  1   1   -3  1   0   -1  1   -2  -4  0   0   0   
R   -1  -4  0   1   -1  -2  1   0   0   -2  -1  0   0   0   1   
S   1   -3  0   -1  -1  0   2   -2  0   -2  -2  2   1   -1  2   1   
T   2   2   -2  -1  -2  -1  1   0   0   1   1   -1  0   0   0   0   1   
V   -1  1   -1  -2  1   -1  -2  1   -2  2   1   -8  0   -2  0   -1  1   1   
W   -1  -1  -1  -8  1   -12 4   1   -3  1   -6  -10 -16 1   -2  -2  0   -1  7   
Y   -2  1   -2  -8  3   -2  1   -1  0   0   -3  1   1   1   1   1   -2  0   3   2   
X   1   -6  -8  3 

As a reminder, here is BLOSUM 62 :
```
A 4 
R -1   5 
N -2   0   6 
D -2  -2   1   6 
C  0  -3  -3  -3   9 
Q -1   1   0   0  -3   5 
E -1   0   0   2  -4   2   5 
G  0  -2   0  -1  -3  -2  -2   6 
H -2   0   1  -1  -3   0   0  -2   8 
I -1  -3  -3  -3  -1  -3  -3  -4  -3   4 
L -1  -2  -3  -4  -1  -2  -3  -4  -3   2   4 
K -1   2   0  -1  -3   1   1  -2  -1  -3  -2   5 
M -1  -1  -2  -3  -1   0  -2  -3  -2   1   2  -1   5 
F -2  -3  -3  -3  -2  -3  -3  -3  -1   0   0  -3   0   6 
P -1  -2  -2  -1  -3  -1  -1  -2  -2  -3  -3  -1  -2  -4   7 
S  1  -1   1   0  -1   0   0   0  -1  -2  -2   0  -1  -2  -1   4 
T  0  -1   0  -1  -1  -1  -1  -2  -2  -1  -1  -1  -1  -2  -1   1   5 
W -3  -3  -4  -4  -2  -2  -3  -2  -2  -3  -2  -3  -1   1  -4  -3  -2  11 
Y -2  -2  -2  -3  -2  -1  -2  -3   2  -1  -1  -2  -1   3  -3  -2  -2   2   7 
V  0  -3  -3  -3  -1  -2  -2  -3  -3   3   1  -2   1  -1  -2  -2   0  -3  -1   4 
B -2  -1   3   4  -3   0   1  -1   0  -3  -4   0  -3  -3  -2   0  -1  -4  -3  -3   4
Z -1   0   0   1  -3   3   4  -2   0  -3  -3   1  -1  -3  -1   0  -1  -3  -2  -2   1   4
X  0  -1  -1  -1  -2  -1  -1  -1  -1  -1  -1  -1  -1  -1  -2   0   0  -2  -1  -1  -1  -1  -1
   A   R   N   D   C   Q   E   G   H   I   L   K   M   F   P   S   T   W   Y   V   B   Z   X
```

Values are very different, which is normal since this matrix was built for a small, specific subset of the BLOCKS database and with a different required identity. Diagonal values also differ significantly. New substitutions are allowed (like between P and A), other substitutions no longer are (like all substitutions to and from Z, which is not present in the SH3 or PDZ domains).

Let's try to do some alignments with these BLOSUM matrices :

In [7]:
blosum62 = Score("blosum62.iij", "BLOSUM 62")
a = AlignMatrix(blosum62)
b = AlignMatrix(blosum40pdz)
c = AlignMatrix(blosum70pdz)
sequences = [seq for seq in loadFasta("PDZ-sequences.fasta")]

for alignM in (a,b,c):
    for align in a.globalAlign(sequences[0], sequences[1], -12, -2, False):
        align.condensed = True
        print(align)
        break

ValueError: Could not find amino acid name 